In [1]:
import torch
import os

data = 'cut.mp4'

# Загружаем натренированную модель
model = torch.hub.load(os.path.abspath("") + '/', 'custom', path=os.path.abspath("")+'/runs/train/exp20/weights/best.pt', source='local')
model.conf = 0.1

YOLOv5 🚀 6c32967 torch 1.8.0+cu111 CUDA:0 (NVIDIA GeForce GTX 1050 Ti, 4037MiB)

Fusing layers... 
Model Summary: 213 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [2]:
import cv2
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display
import pandas
import numpy as np
import json
import math


def get_avg(r):
    result = 0
    for i in range(len(r)):
        result = result + (r[i][2] - r[i][0])
    return result / max(0, len(r))

def get_wind_coeff():
    return 1

def get_fire_radius(distance, screen_width, fire_width):
    meta_camera_matrix = 8.46
    meta_camera_focal = 120
    popravka = 25
    fire_meter_width = popravka * 2 * distance * math.tan((fire_width / screen_width) * 2 * math.atan(meta_camera_matrix / (2 * meta_camera_focal)))
    return fire_meter_width / 2
    # return get_wind_coeff() * (fire_meter_width ** 2) * math.pi / 10000

def get_fire_area(distance, screen_width, fire_width):
    return get_wind_coeff() * (get_fire_radius(distance, screen_width, fire_width) ** 2) * math.pi / 10000 / 2
def compute_point_perspective_transformation(matrix,list_downoids):
    list_points_to_detect = np.float32(list_downoids).reshape(-1, 1, 2)
    transformed_points = cv2.perspectiveTransform(list_points_to_detect, matrix)
    transformed_points_list = []
    for i in range(0,transformed_points.shape[0]):
        transformed_points_list.append([transformed_points[i][0][0],transformed_points[i][0][1]])
    return transformed_points_list

def order_points(rect):
    rect = np.zeros((4, 2), dtype = "float32")

    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    
    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    return rect


def four_point_transform(image, pts):
    rect = order_points(pts)
    (tl, tr, br, bl) = rect

    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))

    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype = "float32")
    
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    warped = cv2.warpPerspective(warped, M, (maxWidth, maxHeight))
    
    return warped, M

In [3]:
cap = cv2.VideoCapture(data)
frame_count = 0
array_groundpoints = []
fire_areas = []
dists = []

fps = cap.get(cv2.CAP_PROP_FPS)

fourcc= cv2.VideoWriter_fourcc(*'XVID')
out= cv2.VideoWriter('outcut.avi',fourcc, 25.0, (1280, 720)) 

while cap.isOpened():
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
  
    # читаем кадр из видео
    ret, im = cap.read()
    
    if ret == True:
        frame_count += 1
        result = Image.fromarray(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))

        # отдаем кадр в модель для детекции
        results = model(result)

        # выводим значения детекции
        # также рисуем bounding box на основе полученных точек
        if not results.pandas().xyxy[0].empty:
            isAlert = False
            #print(results.pandas().xyxy[0])
            r = results.pandas().xyxy[0].to_numpy()
            for line in r:
                isAlert = True
                img1 = ImageDraw.Draw(result)  
                img1.rectangle((line[0], line[1], line[2], line[3]), outline ="red")
                array_groundpoints.append(((line[0] + line[3])/2, (line[1] + line[2])/2))
                    
            # получаем примерный вид с высоты птичьего полета
            # с помощью cv2.getPerspectiveTranform()
            horizonline = 140
            calculatedangle = 75
            height, width = im.shape[:2]
            pts = np.array([
                [calculatedangle, horizonline],
                [width - calculatedangle, horizonline],
                [0, height],
                [width, height]], dtype = "float32")

            warped, matrix = four_point_transform(im, pts)
            transformed_downoids = compute_point_perspective_transformation(matrix, array_groundpoints)

            if len(transformed_downoids) >= 1:
                halfW = width / 2
                dist = np.sqrt((float(transformed_downoids[0][0]) - halfW) ** 2 + (height - float(transformed_downoids[0][1])) ** 2)
                dists.append(dist)
                fire_area = get_fire_area(dist, width, get_avg(r))
                filename = str(frame_count)
                #print("distance: ", dist)
                #print("whole_fire_area: ", fire_area, "га")

                for i in range(len(r)):
                    fire_radius = get_fire_radius(dist, width, r[i][2] - r[i][0])
                    fire_areas.append((i, fire_radius))
                    #print(i, " ", "x:", transformed_downoids[i][0], "y:", transformed_downoids[i][1], " | damage_radius: ", fire_radius)

                data_to_json = {
                        "frame" : filename,
                        "isAlert": str(isAlert),
                        "dist": str(dist),
                        "area:": str(fire_area)
                }
                
                with open("data_file.json", "w+") as write_file:
                     json.dump(data_to_json, write_file)
                array_groundpoints.clear()
                
            font = ImageFont.truetype('arial.ttf', 24) 
            img1.text((5, 5), "Найдено возгораний: " + str(len(transformed_downoids)), font = font, align ="left", fill=(255,0,0,255))
            transformed_downoids.clear()
            if len(dists) > 0:
                img1.text((5, 35), "Дистанция до первого очага (в у.е): " + '{0:.3f}'.format(dists[0]), font = font, align ="left", fill=(255,0,0,255))
                img1.text((5, 65), "Площадь возгорания первого очага (в у.е): " + '{0:.3f}'.format(fire_areas[0][1]), font = font, align ="left", fill=(255,0,0,255))
            dists.clear()
            fire_areas.clear()
                
        out.write(cv2.cvtColor(np.array(result), cv2.COLOR_RGB2BGR))
    else:
        break
        
cap.release()
cv2.destroyAllWindows()